# Data clening

In [ ]:
import json
with open("delfi.json", "r") as read_file:
    data = json.load(read_file)

In [ ]:
cleaned_data = []
for d in data:
    if d["categorys"] in ("projektai", "m360") or len(d["text"]) < 1000:# d["text"] == "": #len(d["text"]) < 1000:
        continue
    elif d["categorys"] == 'sportas':
        d["categorys"] = 'Sportas'
    elif d["categorys"][0].startswith('DELFI '):
        d["categorys"] = d["categorys"][0][6:]
    elif isinstance(d["categorys"], list):
        d["categorys"] = d["categorys"][0]
    cleaned_data.append(d)
cleaned_data = [d for d in cleaned_data if d["categorys"] in ("Verslas", "Mokslas", "Veidai", "Auto", "Sportas")]
print("From {} to {}.".format(len(data), len(cleaned_data)))

# Preprocessing

In [ ]:
def stemmer(tokens):
    with open("tokens.txt", "w") as token_file:
        token_file.write("\n".join(tokens))
    args = ("./stemwords", "-l", "lt", "-i", "tokens.txt", "-o", "stems.txt")
    popen = subprocess.Popen(args, stdout=subprocess.PIPE)
    popen.wait()
    with open("stems.txt", "r") as stem_file:
        stems = stem_file.read().split("\n")
    return stems

In [ ]:
%%time
import re
import subprocess

text_file = open("Lithuanian stop words", "r")
stopwords = text_file.read().split("\n")
text_file.close()

for d in cleaned_data:    
    # tokenize & lowercase
    tokens = re.sub("[\W\d_]+", " ", d["text"]).lower().split() # ka daryti su 1992-ųjų, romėniškais skaičiais, 2 mln. eur\
    
    # separate stop words
    stop_tokens = [words for words in tokens if words     in stopwords]
    d["stop_tokens"] = stop_tokens
    tokens      = [words for words in tokens if words not in stopwords]
    d["tokens"]      = tokens
    
    # steams
    d["stems"]      = stemmer(tokens)
    d["stop_stems"] = stemmer(stop_tokens)    

# TODO add stats    
    
# save as file
with open("delfi_pre.json", "w") as write_file:
    json.dump(cleaned_data, write_file)

In [ ]:
# This page has pure from for lemmatization data http://donelaitis.vdu.lt/main_helper.php?id=4&nr=7_2
# I fill try to forge http request to get file.
# Becouse tool returns invalid xml I need to parse it myself.
from bs4 import BeautifulSoup
import requests
import re
import time

def lemmer(text):
    textload = text
    payload = {'tekstas': textload,
               'tipas': 'anotuoti',
               'pateikti': 'LM',
               'veiksmas': 'Rezultatas puslapyje'}
    while True:
        try:
            r = requests.post('http://donelaitis.vdu.lt/main_helper.php?id=4&nr=7_2',
                             data = payload, allow_redirects=True)
        
            if (r.status_code != requests.codes.ok):
                raise Exception(r.status_code)

            soup = BeautifulSoup(r.text).decode(formatter=None) # to remove &lt
            #return  str(soup)
            textAnalysis = re.sub("<html>[\w\W]*</form>|<br>\n<p/><br>\n|(\n)?</body></html>|<br>", "", str(soup))
            if (len(textAnalysis) == len(text) or len(textAnalysis) == 0):
                raise Exception(textAnalysis)

            return textAnalysis

        except Exception as e:
            print(time.ctime())
            print(e)
            time.sleep(60)
            continue

In [ ]:
# If uncommented will send requests to internet for analysis

#for d in cleaned_data:
#     lemmas = lemmer(d["text"])
#     d["lemms"] = lemmas
#     time.sleep(5)
    
# with open("delfi_lemmas.json", "w") as write_file:
#     json.dump(cleaned_data, write_file)